***
# GPT-2<br>
***

***
## 1 Load the train/val/test/balanced test datasets
Our datasets have been created using the make_datasets notebook. These datasets consist of a training and validation datasets with varying numbers of records, and two test datasets consisting of an imbalanced dataset that keeps the proportions of the emotions of the original dataset, and a balanced test set that comes from drawing the same number of records for each emotion class from the imbalanced test dataset.<br><br>
Citation:<br>
https://towardsdatascience.com/guide-to-fine-tuning-text-generation-models-gpt-2-gpt-neo-and-t5-dc5de6b3bc5e<br>
http://mohitmayank.com/a_lazy_data_science_guide/natural_language_processing/GPTs/#finetuning-gpt-2-for-sentiment-classification<br>


In [1]:
import pandas as pd
import numpy as np

# mount google drive
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# read from the google drive
project_folder = '/content/gdrive/MyDrive/266_Final_Project_Folder'

version = '_17k_5k'

train_source = project_folder + '/datasets/train_data_imbalanced_stratified_no_dupe' + version + '.csv'
test_source = project_folder + '/datasets/test_data_imbalanced_stratified_no_dupe.csv'
val_source = project_folder + '/datasets/validation_data_imbalanced_stratified_no_dupe' + version + '.csv'
balanced_test_source = project_folder + '/datasets/test_data_balanced_no_dupe.csv'

# Load the dfs
train_df = pd.read_csv(train_source)
test_df = pd.read_csv(test_source)
val_df = pd.read_csv(val_source)
balanced_test_df = pd.read_csv(balanced_test_source)

# drop the tweet_id column
train_df.drop(columns=train_df.columns[0], axis=1, inplace=True)
test_df.drop(columns=test_df.columns[0], axis=1, inplace=True)
val_df.drop(columns=val_df.columns[0], axis=1, inplace=True)
balanced_test_df.drop(columns=balanced_test_df.columns[0], axis=1, inplace=True)

### Confirm no overlapping records 

In [3]:
# Check if there are any records in train_df that are in val_df
overlap = train_df['text'].isin(val_df['text'])
print(f"Number of overlapping records, train-val: {overlap.sum()}")

# Check if there are any records in val_df that are in train_df
overlap = val_df['text'].isin(train_df['text'])
print(f"Number of overlapping records, val-train: {overlap.sum()}")

# Check if there are any records in train_df that are in test_df
overlap = train_df['text'].isin(test_df['text'])
print(f"Number of overlapping records, train-test: {overlap.sum()}")

# Check if there are any records in test_df that are in train_df
overlap = test_df['text'].isin(train_df['text'])
print(f"Number of overlapping records, test-train: {overlap.sum()}")

# Check if there are any records in train_df that are in balanced_test_df
overlap = train_df['text'].isin(balanced_test_df['text'])
print(f"Number of overlapping records, train-balanced test: {overlap.sum()}")

# Check if there are any records in train_df that are in balanced_test_df
overlap = balanced_test_df['text'].isin(train_df['text'])
print(f"Number of overlapping records, balanced test, train: {overlap.sum()}")

Number of overlapping records, train-val: 0
Number of overlapping records, val-train: 0
Number of overlapping records, train-test: 0
Number of overlapping records, test-train: 0
Number of overlapping records, train-balanced test: 0
Number of overlapping records, balanced test, train: 0


In [4]:
train_df.shape

(17082, 2)

In [6]:
val_df.shape

(5125, 2)

In [7]:
test_df.shape

(37961, 2)

In [8]:
balanced_test_df.shape

(6020, 2)

***
## 2 Tokenizer

### Find the top word count (frequency > 10)
We'll use this number as our vocabulary size

In [ ]:
# Split the text column into a list of words
words_df = train_df['text'].str.split().to_frame('words')

# Explode the list of words into separate rows
words_df = words_df.explode('words')

# Group by the words and count their occurrences
words_df = words_df.groupby('words').size().reset_index(name='count')
words_df

words  count
0                     !      2
1                    !!      1
2                  !!my      1
3              #cycling      1
4                   #fb      2
...                 ...    ...
17247               zyl      1
17248                 |      1
17249                 ~      1
17250  ï¿½WHYYY????????      1
17251           ï¿½why?      1

[17252 rows x 2 columns]

In [ ]:
# find the words with 'count' > 10
condition = words_df['count'] > 10

# sort the words based on count
new_words_df = words_df[condition].sort_values('count', ascending=False)
new_words_df

words  count
7761           i  26872
6013        feel  11453
1256         and   9986
15479         to   9426
15264        the   8784
...          ...    ...
3323     clients     11
13123       rush     11
8149   instantly     11
8093       inner     11
11042      pains     11

[2056 rows x 2 columns]

In [ ]:
# how many words are > 10?
top_word_count = len(new_words_df)
print('Vocabulary size:', top_word_count)

Vocabulary size: 2056


### Let's tokenize the train tweets!

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=top_word_count, oov_token='<UNK>')
tokenizer.fit_on_texts(train_df['text']) # pass in tweets from training set

In [ ]:
# this is what the tweet looks like as words
train_df['text'][0]

'i feel so repressed with this one now'

In [ ]:
# this is what the tokenized version of the tweet looks like
tokenizer.texts_to_sequences([train_df['text'][0]])

[[2, 3, 18, 1479, 24, 25, 66, 57]]

In [ ]:
import matplotlib.pyplot as plt

# check to see what lenghts of the tweets in the dataset
lengths = [len(t.split(' ')) for t in train_df['text']]
plt.hist(lengths, bins=len(set(lengths)))

(array([2.000e+01, 5.200e+01, 3.150e+02, 5.290e+02, 5.850e+02, 6.630e+02,
        6.860e+02, 6.890e+02, 6.670e+02, 6.710e+02, 7.120e+02, 1.311e+03,
        6.880e+02, 6.780e+02, 6.480e+02, 6.260e+02, 6.030e+02, 5.450e+02,
        5.290e+02, 4.620e+02, 4.560e+02, 4.320e+02, 7.680e+02, 3.060e+02,
        3.230e+02, 2.940e+02, 2.580e+02, 2.400e+02, 2.290e+02, 2.260e+02,
        1.820e+02, 1.770e+02, 1.690e+02, 2.620e+02, 1.150e+02, 1.100e+02,
        1.040e+02, 8.800e+01, 7.100e+01, 6.900e+01, 6.700e+01, 5.400e+01,
        6.200e+01, 4.900e+01, 7.400e+01, 3.700e+01, 2.400e+01, 3.600e+01,
        2.700e+01, 2.000e+01, 2.000e+01, 1.800e+01, 1.500e+01, 4.000e+00,
        5.000e+00, 6.000e+00, 1.000e+00, 2.000e+00, 1.000e+00, 1.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        1.000e+00]),
 array([ 1.        ,  2.08955224,  3.17910448,  4.26865672,  5.35820896,
         6.44776119,  7.53731343,  8.62686567,  9.71641791, 10.80597015,
        11.89552239

In [ ]:
import statistics

# Apply the lambda function to the 'content' column and list called word_count_list'
word_count_list = list(train_df['text'].apply(lambda x: len(x.split(' '))))

# display the maximum and minimum lengths
print('Maximum word count:', max(word_count_list))
print('Minimum word count:', min(word_count_list))
print('Mean word count:', statistics.mean(word_count_list))
print('Median word count:', statistics.median(word_count_list))
print('75th percentile word count:', np.percentile(word_count_list, 75))
print('85th percentile word count:', np.percentile(word_count_list, 85))
print('95th percentile word count:', np.percentile(word_count_list, 95))

Maximum word count: 74
Minimum word count: 1
Mean word count: 19.034656363423487
Median word count: 17.0
75th percentile word count: 25.0
85th percentile word count: 31.0
95th percentile word count: 41.0


The max number of words is 101. We ultimately decided that we are going to keep all the words in the tweet. BERT can handle more than 200 so setting max_len = 200

# Importing packages needed for GPT-2

In [10]:
!pip install transformers # already satisfied, don't really need this

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.3 MB/s eta 0:00:00


In [11]:
import re
import torch
import random
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, GPT2Config, GPT2Model

# Defining classe needed for this notebook

In [ ]:
# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, txt_list, emotion_list, tokenizer, max_length):
        # define variables    
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        self.emotion_label = []
        # map_label = {0: 'sadness', 1: 'anger', 2: 'fear', 3: 'surprise', 4: 'joy', 5: 'love', 6: 'neutral'} #{0:'negative', 4: 'positive'} #
        class_to_index_dict = {'sadness': 0, 'anger': 1, 'fear': 2, 'surprise': 3, 'joy': 4, 'love': 5, 'neutral': 6}
        # iterate through the dataset
        for txt, emotion in zip(txt_list, emotion_list):
            # prepare the text
            prep_txt = f'<|startoftext|>Tweet: {txt}<|pad|>Sentiment: {emotion}<|endoftext|>' # {map_label[emotion]}<|endoftext|>'
            # tokenize
            encodings_dict = tokenizer(prep_txt, truncation=True,
                                       max_length=max_length, padding="max_length")
            # append to list
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            self.labels.append(torch.tensor(class_to_index_dict[emotion])) # (map_label[label])
            self.emotion_label.append(emotion) # (map_label[label])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx], self.labels[idx], self.emotion_label[idx]

# Load mode and tokenizer

### Make sure GPU and CUDA are available

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name ()

'/device:GPU:0'

In [ ]:
print (torch.version.cuda)

11.6


In [ ]:
!pip show torch

Name: torch
Version: 1.13.1+cu116
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.9/dist-packages
Requires: typing-extensions
Required-by: fastai, torchaudio, torchtext, torchvision


In [ ]:
!nvidia-smi

Tue Mar 14 04:07:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    29W /  70W |    333MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.cuda.device_count()

1

### Load model and tokenizer

In [12]:
# using gpt2 (the small model)
model_name = "gpt2"

# keep the seed the same for reliability of results
torch.manual_seed(11)

# load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel .from_pretrained(model_name).cuda()
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50259, 768)

In [13]:
tokenizer("Hello world")['input_ids']

[15496, 995]

In [ ]:
max_length = 200
train_dataset = SentimentDataset(train_df['text'].tolist(), train_df['emotions'].tolist(), tokenizer, max_length=max_length)
val_dataset = SentimentDataset(val_df['text'].tolist(), train_df['emotions'].tolist(), tokenizer, max_length=max_length)

In [ ]:
training_args = TrainingArguments(output_dir='results', num_train_epochs=2, logging_steps=10,
                                 load_best_model_at_end=True, save_strategy="epoch", evaluation_strategy="epoch",
                                 per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                 warmup_steps=100, weight_decay=0.01, logging_dir='logs')

In [ ]:
def data_collator_fun(data):
    input_ids = torch.stack([f[0] for f in data])
    attention_mask = torch.stack([f[1] for f in data])
    labels = torch.stack([f[2] for f in data])
    # emotion_label = torch.stack([f[3] for f in data])
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels} #, 'emotion_label': emotion_label}

In [ ]:
train_dataset.input_ids

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
train_dataset.labels

[0,
 4,
 0,
 4,
 4,
 2,
 1,
 4,
 5,
 4,
 2,
 0,
 4,
 0,
 2,
 0,
 4,
 0,
 0,
 0,
 4,
 1,
 1,
 0,
 1,
 0,
 4,
 4,
 4,
 1,
 0,
 1,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 4,
 4,
 2,
 0,
 0,
 5,
 0,
 5,
 1,
 4,
 1,
 4,
 2,
 0,
 0,
 4,
 2,
 5,
 0,
 0,
 3,
 5,
 1,
 0,
 2,
 4,
 1,
 1,
 4,
 0,
 0,
 0,
 0,
 2,
 4,
 4,
 4,
 5,
 0,
 0,
 0,
 1,
 1,
 4,
 4,
 2,
 0,
 1,
 4,
 0,
 0,
 0,
 4,
 2,
 3,
 0,
 4,
 3,
 4,
 0,
 3,
 2,
 4,
 5,
 4,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 4,
 1,
 4,
 0,
 0,
 2,
 4,
 3,
 2,
 4,
 4,
 3,
 0,
 0,
 4,
 2,
 2,
 4,
 4,
 2,
 4,
 1,
 0,
 4,
 0,
 4,
 4,
 4,
 4,
 2,
 1,
 1,
 5,
 4,
 0,
 1,
 2,
 5,
 4,
 2,
 4,
 1,
 2,
 0,
 0,
 4,
 5,
 1,
 4,
 4,
 0,
 4,
 0,
 0,
 2,
 4,
 4,
 0,
 4,
 2,
 4,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 5,
 4,
 1,
 4,
 5,
 4,
 1,
 0,
 4,
 1,
 5,
 1,
 5,
 0,
 4,
 4,
 0,
 1,
 0,
 4,
 3,
 0,
 4,
 2,
 0,
 6,
 6,
 0,
 2,
 4,
 4,
 4,
 2,
 1,
 5,
 4,
 0,
 0,
 2,
 4,
 1,
 4,
 4,
 4,
 5,
 4,
 5,
 0,
 3,
 1,
 3,
 0,
 4,
 0,
 2,
 4,
 4,
 1,
 4,
 2,
 1,
 0,
 1,
 4,
 5,
 4,
 1,
 4,
 1,
 0,
 0,


In [ ]:
train_dataset.attn_masks

[tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0

In [ ]:
# train_dataset = SentimentDataset(train_df['text'], train_df['emotions'], tokenizer, max_length=512) # might need to change max_length to be the same as all the other model
int_dict = {'sadness': 0, 'anger': 1, 'fear': 2, 'surprise': 3, 'joy': 4, 'love': 5, 'neutral': 6}
y_test_ints = list(map(lambda x: int_dict[x], test_df['emotions'].tolist()))
y_test_ints[:10]

[1, 4, 0, 4, 4, 0, 4, 5, 0, 1]

In [ ]:
test_dataset = (test_df['text'].tolist(), y_test_ints)

In [ ]:
test_dataset[0]

['i feel impatient to get moving forward',
 'i have decided to open wb to a few thoughtful bloggers who i feel have something really valuable to walking barefoot readers',
 'i resolved at that moment to do my bit when i get back home for anyone i know who has come down from another country and occasionally feels a little lost',
 'i like his tip especially if you need assistance in feeling satisfied and overcoming cravings',
 'i feel that if he did not want to be on the show he would have said so and i believe his parents love him enough that they would have respected his wishes',
 'i feel disheartened right after',
 'i have the feeling they can relate with me in most things in those things which are important for me',
 'i never orgasm the feeling is lovely',
 'i on career i feel like lebron without the ring youtube target blank yo gotti on career i feel like lebron without the ring youtube',
 'im on the slide feeling so despised',
 'i already feel bad because i am finding myself rootin

In [ ]:
test_dataset[1]

[1,
 4,
 0,
 4,
 4,
 0,
 4,
 5,
 0,
 1,
 0,
 4,
 2,
 3,
 1,
 0,
 0,
 1,
 0,
 4,
 0,
 6,
 0,
 0,
 4,
 5,
 4,
 4,
 2,
 1,
 4,
 6,
 0,
 4,
 1,
 0,
 1,
 4,
 0,
 1,
 4,
 2,
 0,
 4,
 0,
 1,
 4,
 4,
 4,
 0,
 2,
 0,
 5,
 1,
 3,
 5,
 4,
 5,
 1,
 4,
 2,
 4,
 0,
 1,
 4,
 0,
 0,
 4,
 0,
 4,
 2,
 4,
 2,
 1,
 2,
 4,
 4,
 2,
 1,
 0,
 0,
 2,
 1,
 2,
 4,
 0,
 4,
 4,
 0,
 1,
 0,
 4,
 0,
 2,
 4,
 0,
 0,
 1,
 0,
 4,
 2,
 0,
 5,
 5,
 4,
 5,
 0,
 4,
 0,
 4,
 4,
 2,
 3,
 6,
 3,
 3,
 4,
 1,
 2,
 0,
 4,
 0,
 5,
 2,
 0,
 4,
 2,
 4,
 4,
 4,
 4,
 4,
 0,
 4,
 2,
 4,
 0,
 4,
 0,
 4,
 5,
 4,
 5,
 1,
 4,
 0,
 0,
 1,
 4,
 6,
 0,
 4,
 4,
 4,
 0,
 4,
 1,
 4,
 1,
 5,
 1,
 0,
 4,
 1,
 0,
 0,
 6,
 4,
 2,
 0,
 5,
 0,
 4,
 0,
 1,
 0,
 4,
 4,
 0,
 4,
 4,
 4,
 5,
 1,
 4,
 4,
 4,
 2,
 4,
 0,
 4,
 6,
 0,
 1,
 0,
 4,
 5,
 0,
 0,
 1,
 0,
 4,
 4,
 1,
 2,
 5,
 2,
 6,
 4,
 5,
 0,
 5,
 1,
 4,
 0,
 4,
 4,
 2,
 4,
 0,
 0,
 0,
 4,
 4,
 0,
 4,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 4,
 4,
 2,
 1,
 0,
 0,
 2,
 4,
 4,
 4,
 4,
 1,
 1,
 0,
 2,
 0,
 3,


# Model training

In [ ]:
# creating training arguments
training_args = TrainingArguments(output_dir='results', num_train_epochs=2, logging_steps=10,
                                 load_best_model_at_end=True, save_strategy="epoch", evaluation_strategy="epoch",
                                 per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                 warmup_steps=100, weight_decay=0.01, logging_dir='logs')

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def data_collator_func(data):
    input_ids = torch.stack([f[0] for f in data])
    attn_masks = torch.stack([f[1] for f in data])
    labels = torch.tensor([f[2] for f in data])  # convert integer labels to tensor
    return {'input_ids': input_ids, 'attention_mask': attn_masks, 'labels': labels}

In [ ]:
# start training
Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset,
        data_collator=data_collator_func).train()
        # data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
        #                             'attention_mask': torch.stack([f[1] for f in data]),
        #                             'labels': torch.stack([f[2] for f in data])}).train()

***** Running training *****
  Num examples = 17082
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 17082


ValueError: ignored

# Test the model

In [ ]:
_ = model.eval()

# run model inference on all test data
original_label, predicted_label, original_text, predicted_text = [], [], [], []
# map_label = {0:'negative', 4: 'positive'}
# iter over all of the test data
for text, label in tqdm(zip(test_dataset[0], test_dataset[1])):
    # create prompt (in compliance with the one used during training)
    prompt = f'<|startoftext|>Tweet: {text}\nSentiment:'
    # generate tokens
    generated = tokenizer(f"{prompt}", return_tensors="pt").input_ids.cuda()
    # perform prediction
    sample_outputs = model.generate(generated, do_sample=False, top_k=50, max_length=512, top_p=0.90, 
            temperature=0, num_return_sequences=0)
    # decode the predicted tokens into texts
    predicted_text  = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    # extract the predicted sentiment
    try:
        pred_sentiment = re.findall("\nSentiment: (.*)", predicted_text)[-1]
    except:
        pred_sentiment = "None"
    # append results
    original_label.append(map_label[label])
    predicted_label.append(pred_sentiment)
    original_text.append(text)
    predicted_text.append(pred_text)

# transform result into dataframe
df = pd.DataFrame({'original_text': original_text, 'predicted_label': predicted_label, 
                    'original_label': original_label, 'predicted_text': predicted_text})

# predict the accuracy
print(f1_score(original_label, predicted_label, average='macro'))

***
## 3 Padding and truncating the training data sequences to make sure that the shape is the same for each string.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len = 200 # set the largest tweet size, >200 words -> truncate, <200 words -> pad

def get_sequences(tokenizer, tweets):
  sequences = tokenizer.texts_to_sequences(tweets)
  padded = pad_sequences(sequences, truncating='post', padding='post', maxlen=max_len) # truncating happens at the end of the sequence (post)
  return padded # all examples will be the returned with same input size

In [ ]:
padded_train_seq = get_sequences(tokenizer, train_df['text'])
padded_train_seq[0]

***
## 4 Convert labels to numeric

### Function to convert text labels to numeric (reuse for all datasets)

In [ ]:
# keeping this consistent for all models
class_to_index_dict = {'sadness': 0, 'anger': 1, 'fear': 2, 'surprise': 3, 'joy': 4, 'love': 5, 'neutral': 6}
class_to_index_dict

In [ ]:
def names_to_ids(class_to_index, data):
  return np.array([class_to_index.get(x) for x in data])

In [ ]:
# confirming if it worked (change index) and they should line up with above dict
index = 7
train_labels = names_to_ids(class_to_index_dict, train_df['emotions'])
print('train_df.iloc[7]:\n', train_df.iloc[index])
print('numeric label:', train_labels[index])

***
## 5 Create the model

In [ ]:
%matplotlib inline

import tensorflow as tf
import matplotlib.pyplot as plt
import random

def show_history(h):
    epochs_trained = len(h.history['loss'])
    plt.figure(figsize=(16, 6))

    plt.subplot(1, 2, 1)
    plt.plot(range(0, epochs_trained), h.history.get('accuracy'), label='Training')
    plt.plot(range(0, epochs_trained), h.history.get('val_accuracy'), label='Validation')
    plt.ylim([0., 1.])
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(0, epochs_trained), h.history.get('loss'), label='Training')
    plt.plot(range(0, epochs_trained), h.history.get('val_loss'), label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
print('Using TensorFlow version', tf.__version__)

In [ ]:
# ref: https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/
# ref: https://analyticsindiamag.com/complete-guide-to-bidirectional-lstm-with-python-codes/
# ref: https://stackoverflow.com/questions/40331510/how-to-stack-multiple-lstm-in-keras

# keep the vocabulary size the same as the base model, 11039
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=11039, output_dim=8, input_length=max_len),
    
    # the RNN part of the model
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(10, return_sequences=True)), # since using an LSTM layer inside a bidrectional layer, context can go in either direction; return_sequences=True means at every time step, there is an output
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(10, )),

    tf.keras.layers.Dense(7, activation='softmax') # 7 for the seven classes

])

model.compile(
    loss='sparse_categorical_crossentropy', # since this is a multiclass classification problem, use categorical crossentropy; since classes are integers (not one hot encoded/binary), use sparse (just like in A3)
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
model.summary()

***
## 6 Train the model

### Convert the labels of the validation set to numeric (just like the training set)

In [ ]:
# reminder of the class_to_index mapping
class_to_index_dict

In [ ]:
val_seq = get_sequences(tokenizer, val_df['text'])
val_labels = names_to_ids(class_to_index_dict, val_df['emotions'])

In [ ]:
# checking out the first input in validation sequence
val_seq[0], val_df['text'][0], val_df['emotions'][0]

In [ ]:
print('val_df.iloc[0]:\n', val_df.iloc[0])
print('numeric label:', val_labels[0])

#### Check out the shape to make sure they're as expected

In [ ]:
padded_train_seq.shape

In [ ]:
train_labels.shape

In [ ]:
val_seq.shape

In [ ]:
val_labels.shape

### From Natalie's Notebook
https://github.com/datasci-w266/2023-spring-main/blob/master/materials/walkthrough_notebooks/keras_with_limited_ram/keras_training_with_limited_ram.ipynb<br><br>
https://edstem.org/us/courses/32607/discussion/2699088?comment=6229916

#### Inspired by Natalie's notebook, made my own generator that chunks up my data into pieces that won't make my machine run out of memory before completing one epoch
If generator is not used, might delete this section

In [ ]:
# from sklearn.utils.random import sample_without_replacement

# # Define a generator function
# def generator(data, labels, batch_size):
#     # num_samples = len(data)
#     # while True:
#     #     indices = np.random.randint(num_samples, size=batch_size)
#     #     yield data[indices], labels[indices]

#     # Loop over batches
#     for i in range(0, len(data), batch_size):
      
#       # Get a batch of indices using sample_without_replacement()
#       batch_indices = sample_without_replacement(n_population=len(data), n_samples=batch_size)
      
#       # Yield a batch of features and labels using the indices
#       yield data[batch_indices], labels[batch_indices]

In [ ]:
# Create the generator
# batch_size = 1 # using this batch size because that's what seemed reasonable based on the number of records in the training dataset
# gen = generator(data=padded_train_seq, labels=train_labels, batch_size=batch_size)

#### Setting up checkpoints

In [ ]:
# reference: https://pynative.com/python-timezone/
# make a timestamp so it can be part of the folder name
# otherwise, all the checkpoints get dumped into the same
# folder and that's not great
import datetime
import pytz

local_tz = pytz.timezone('America/Denver')
local_dt = datetime.datetime.now(local_tz)

# Format the datetime object as a string without spaces
timestamp = local_dt.strftime('/%Y-%m-%d-%H%M')

# Print the string
print(timestamp)

In [ ]:
!ls '/content/gdrive/MyDrive/266_Final_Project_Folder/cm_working_dir/cm_checkpoints/'

In [ ]:
checkpoint_dir = project_folder + '/cm_working_dir/cm_checkpoints/17_5_model'
checkpoint_filepath = checkpoint_dir + '/weights.{epoch:02d}-{val_accuracy:.2f}.hdf5' #+ timestamp + '/weights.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True)

In [ ]:
# check to make sure that the folder name is as expected
# format: .....cm_checkpoints/YYYY-MM-DD-TIME/....
print(checkpoint_filepath)

### Training the Model!!!

##### Uncomment cell below to load checkpoints; else keep commented to start fresh

In [ ]:
# uncomment below when loading checkpoints
# hdf5_checkpoint = 'weights.05-0.99' # change this to the right filename for the checkpoint
# checkpoint_filepath = checkpoint_dir + '/' + hdf5_checkpoint + '.hdf5'
# model.load_weights(checkpoint_filepath)

In [ ]:
# batch_size = 64
# steps_per_epoch = len(padded_train_seq) / batch_size

# history = model.fit(
#     gen,
#     validation_data=(val_seq, val_labels),
#     steps_per_epoch=steps_per_epoch,
#     epochs=1,
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2), # if model doesn't see improvement in val_accuracy after 2 epochs, it will stop training
#         model_checkpoint_callback
#     ]
# )

##### Training a fresh model (don't run this if running cells above)

In [ ]:
batch_size = 64
steps_per_epoch = len(padded_train_seq) / batch_size

history = model.fit(
    # gen,
    padded_train_seq,
    train_labels,
    validation_data=(val_seq, val_labels),
    steps_per_epoch=steps_per_epoch,
    epochs=5,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2), # if model doesn't see improvement in val_accuracy after 2 epochs, it will stop training
        model_checkpoint_callback
    ]
)

In [ ]:
show_history(history)

***
## 7 Evaluate the model using imbalanced test_df

In [ ]:
# borrowing this from A3 assignment, but turned it into a function
# classification report (cr) and confusion matrix (cm) from A3
from sklearn.metrics import classification_report
import seaborn as sns

def make_cr_cm(preds, labels):

  predictions_model = tf.argmax(preds, axis=-1)
  predictions_model

  # convert labels to string because classification_report needs target_names as str
  # labels are currently ints, but will change to str of in (i.e. '0', '1', etc)
  class_names = list(set([str(digit) for digit in labels]))

  # convert digits in target_names to the actual name of the emotion from the 
  # using class_to_index dictionary (i.e. '0' -> 'love', '5' -> 'neutral', etc.)
  class_names = [list(class_to_index_dict.keys())[list(class_to_index_dict.values()).index(int(name))] for name in class_names]

  print('Classification Report:\n\n', classification_report(labels, predictions_model.numpy(), target_names=class_names))

  print('Confusion Matrix:\n')
  
  # confusion matrix from A3
  cm = tf.math.confusion_matrix(labels, predictions_model)
  cm = cm/cm.numpy().sum(axis=1)[:, tf.newaxis]

  plt.figure(figsize=(20,7))
  sns.heatmap(
      cm, annot=True,
      xticklabels=class_names,
      yticklabels=class_names)
  plt.xlabel("Predicted")
  plt.ylabel("True")
  plt.show()

### Padding the imbalanced test set for evaluation

In [ ]:
padded_test_seq = get_sequences(tokenizer, test_df['text'])
padded_test_seq[0]

In [ ]:
# reminder of the class_to_index mapping
class_to_index_dict

In [ ]:
# confirming if it worked (change index) and they should line up with above dict
index = 7
test_labels = names_to_ids(class_to_index_dict, test_df['emotions'])
print('test_df.iloc[7]:\n', test_df.iloc[index])
print('numeric label:', test_labels[index])

### Evaluating the model using test_df!!!

In [ ]:
model.evaluate(padded_test_seq, test_labels)

In [ ]:
# printing out a random tweet and its label, then calling
# model.predict to see what the model predicts
i = random.randint(0, len(test_labels) - 1)

print('Sentence:', test_df.iloc[i])
print('Emotion:', test_labels[i])

p = model.predict(np.expand_dims(padded_test_seq[i], axis=0))[0]
pred_class = np.argmax(p).astype('uint8')

print('Predicted Emotion:', pred_class)

In [ ]:
preds_imbalanced_test_set = model.predict(padded_test_seq)

In [ ]:
make_cr_cm(preds_imbalanced_test_set, test_labels)

***
## 8 Evaluating the model again, but with balanced_test_df!!!

### Padding the balanced test set for evaluation

In [ ]:
padded_balanced_test_seq = get_sequences(tokenizer, balanced_test_df['text'])
padded_balanced_test_seq[0]

In [ ]:
# reminder of the class_to_index mapping
class_to_index_dict

In [ ]:
# confirming if it worked (change index) and they should line up with above dict
index = 7
balanced_test_labels = names_to_ids(class_to_index_dict, balanced_test_df['emotions'])
print('balanced_test_df.iloc[7]:\n', balanced_test_df.iloc[index])
print('numeric label:', balanced_test_labels[index])

### Evaluating the model using balanced_test_df!!!

In [ ]:
model.evaluate(padded_balanced_test_seq, balanced_test_labels)

In [ ]:
# printing out a random tweet and its label, then calling
# model.predict to see what the model predicts
i = random.randint(0, len(balanced_test_labels) - 1)

print('Sentence:', balanced_test_df.iloc[i])
print('Emotion:', balanced_test_labels[i])

p = model.predict(np.expand_dims(padded_balanced_test_seq[i], axis=0))[0]
pred_class = np.argmax(p).astype('uint8')

print('Predicted Emotion:', pred_class)

In [ ]:
preds_balanced_test_set = model.predict(padded_balanced_test_seq)

In [ ]:
make_cr_cm(preds_balanced_test_set, balanced_test_labels)

In [ ]:
# download packages
#!pip install transformers==4.8.2

# import packages
import re
import torch
import random
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

## Define class and functions
#--------

# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, txt_list, label_list, tokenizer, max_length):
        # define variables    
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        map_label = {0:'negative', 4: 'positive'}
        # iterate through the dataset
        for txt, label in zip(txt_list, label_list):
            # prepare the text
            prep_txt = f'<|startoftext|>Tweet: {txt}<|pad|>Sentiment: {map_label[label]}<|endoftext|>'
            # tokenize
            encodings_dict = tokenizer(prep_txt, truncation=True,
                                       max_length=max_length, padding="max_length")
            # append to list
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            self.labels.append(map_label[label])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]

# Data load function
def load_sentiment_dataset(tokenizer):
    # load dataset and sample 10k reviews.
    file_path = project_folder + "/datasets/training.1600000.processed.noemoticon.csv"
    df = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)
    df = df[[0, 5]]
    df.columns = ['label', 'text']
    df = df.sample(10000, random_state=1)

    # divide into test and train
    X_train, X_test, y_train, y_test = \
              train_test_split(df['text'].tolist(), df['label'].tolist(),
              shuffle=True, test_size=0.05, random_state=1, stratify=df['label'])

    # format into SentimentDataset class
    train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_length=512)

    # return
    return train_dataset, (X_test, y_test)

## Load model and data
#--------

# set model name
model_name = "gpt2"
# seed
torch.manual_seed(42)

# load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel .from_pretrained(model_name).cuda()
model.resize_token_embeddings(len(tokenizer))

# prepare and load dataset
train_dataset, test_dataset = load_sentiment_dataset(tokenizer)

## Train
#--------
# creating training arguments
training_args = TrainingArguments(output_dir='results', num_train_epochs=2, logging_steps=10,
                                 load_best_model_at_end=True, save_strategy="epoch", evaluation_strategy="epoch",
                                 per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                 warmup_steps=100, weight_decay=0.01, logging_dir='logs')

# start training
Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset,
        data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                    'attention_mask': torch.stack([f[1] for f in data]),
                                    'labels': torch.stack([f[2] for f in data])}).train()


loading file https://huggingface.co/gpt2/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/684fe667923972fb57f6b4dcb61a3c92763ad89882f3da5da9866baf14f2d60f.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f
loading file https://huggingface.co/gpt2/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/c0c761a63004025aeadd530c4c27b860ec4ecbe8a00531233de21d865a402598.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/gpt2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/gpt2/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/gpt2/resolve/main/tokenizer_config.json from cache at None
loading file https://huggingface.co/gpt2/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/16a2f78023c8dc511294f0c97b5e10fde3ef9889ad6d11ffaa2a00714e73926e.cf2d0ecb83b6df91b3dbb53f1d1e4c311578b

TypeError: ignored

In [ ]:
# start training
Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset,
        data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                    'attention_mask': torch.stack([f[1] for f in data]),
                                    'labels': torch.stack(torch.tensor([f[2] for f in data]))}).train()

***** Running training *****
  Num examples = 9500
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 9500


ValueError: ignored